In [ ]:
import tensorflow as tf
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"
from tf_keras import losses,optimizers
from tf_keras import models
import importlib

In [3]:
from utils.load_dataset import load_dataset
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

In [4]:
from utils.reformat_image import reformat_image

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples // 4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [56]:

from unsuccessful import OptimalBrainDamage

importlib.reload(OptimalBrainDamage)
from unsuccessful.OptimalBrainDamage import OptimalBrainDamage
model_SBD = models.load_model("../model_experiments/custom_vgg16.keras")

pruning = OptimalBrainDamage(model=model_SBD, sparsity=0.3)

# Générer des cibles (étiquettes) dans la plage [0, 1] pour 2 classes
targets = tf.random.uniform((1,), minval=0, maxval=2, dtype=tf.int32)

# Générer une entrée d'image aléatoire
inputs = tf.random.normal((1, 224, 224, 3))  # Exemple d'entrée (1 image, 224x224x3)

# Calculer la perte à l'intérieur de tf.GradientTape
with tf.GradientTape() as tape:
    tape.watch(model_SBD.trainable_variables)  # Assurez-vous de regarder les variables du modèle
    predictions = model_SBD(inputs)  # Passer les inputs dans le modèle
    print(predictions)
    loss = losses.SparseCategoricalCrossentropy()(targets, predictions)  # Calculer la perte avec les targets

# Calculer les gradients
gradients = tape.gradient(loss, model_SBD.trainable_variables)


pruningmodel = pruning.run(inputs, targets)
pruningmodel.summary()

tf.Tensor([[0.28264362 0.7173563 ]], shape=(1, 2), dtype=float32)
Calcul de la Hessienne...


InvalidArgumentError: Graph execution error:

Detected at node loop_body/AddN_2/pfor/AddN/tmp_var_zeros defined at (most recent call last):
<stack traces unavailable>
Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 150528
  }
  dim {
    size: 1
  }
  dim {
    size: 14
  }
  dim {
    size: 14
  }
  dim {
    size: 512
  }
}

	 [[{{node loop_body/AddN_2/pfor/AddN/tmp_var_zeros}}]] [Op:__inference_get_hessian_127160]

In [ ]:
pruningmodel.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])